In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import time
from sklearn.metrics import accuracy_score as acc
print(tf.version) # tensorflow 2.0
pd.options.display.max_rows = 8 # will use 8 by default for count, mean, std ... max
pd.options.display.max_columns = 9
pd.options.display.float_format = '{:.6f}'.format
pd.set_option('mode.chained_assignment', None)

<module 'tensorflow_core._api.v2.version' from 'c:\\users\\r-k-l\\appdata\\local\\programs\\python\\python37\\lib\\site-packages\\tensorflow_core\\_api\\v2\\version\\__init__.py'>


In [2]:
df = pd.read_csv("t_test_features_5.csv")

# change y in the csv file to be assigned to one of three classes: High-grade, Low-grade, Normal
# for i in range (0,324): # 0 - 323, same size as x
#     if df['class'][i].startswith('High-grade'):  # if the last column contains text "High-grade", etc below.
#         df['class'][i] = 'High-grade'
#     elif df['class'][i].startswith('Low-grade'):
#         df['class'][i] = 'Low-grade'
#     elif df['class'][i].startswith('Normal'):
#         df['class'][i] = 'Normal'


# https://stackoverflow.com/questions/38250710/how-to-split-data-into-3-sets-train-validation-and-test
# x represents attributes, y represents class label
training, validation, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))]) # 60% test, 20% validation, 20% test split.
x_train = training.drop(['class'], axis=1)
y_train = training['class']
x_validation=validation.drop(['class'], axis=1)
y_validation=validation['class']
x_test=test.drop(['class'], axis=1)
y_test=test['class']
# Encode class label y to be 0, 1, or 2
from sklearn.preprocessing import LabelEncoder
lbl_encoder = LabelEncoder()
y_train= lbl_encoder.fit_transform(y_train)
y_test= lbl_encoder.fit_transform(y_test)
y_validation= lbl_encoder.fit_transform(y_validation)
del training, validation, test # clear memory of variables not needed

In [3]:
y_test

array([1, 1, 2, 0, 1, 1, 0, 0, 0, 1, 2, 0, 0, 1, 2, 1, 0, 1, 1, 1, 2, 0,
       0, 1, 2, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 2, 2])

In [4]:
def construct_feature_columns(input_features_DataFrame):
  """Construct the TensorFlow Feature Columns.
  Args:
    input_features: DataFrame with the names of the numerical input features to use.
  Returns:
    A set of tf numeric feature columns
  """
  tensorSet = ([])
  for elem in input_features_DataFrame:
    tensorSet.append(tf.feature_column.numeric_column(key=str(elem), dtype=tf.dtypes.float64) ) # where elem is a str feature label
  return tensorSet

# Create the input function for training + evaluation. boolean = True for training.
def input_fn(features, labels, training=True, batch_size=32 ):
    dataf = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    if training:
        dataf = dataf.shuffle(100).repeat() # shuffle ~<half the data
    return dataf.batch(batch_size=batch_size)

In [5]:
# Construct feature columns, which is a list of tensors with the feature names, shown below. 
feature_columns=construct_feature_columns(df.drop(['class'], axis=1).head(0))

In [6]:
len(feature_columns)

300

In [7]:
# tf.keras.backend.set_floatx('float64')
timer_start = time.time()
learning_rate=0.001
hidden_units=[37,30,19]
print("Building model: Hidden units = " , hidden_units)
model=tf.estimator.DNNClassifier(hidden_units=hidden_units, feature_columns=feature_columns, n_classes=3,
        optimizer=lambda: tf.keras.optimizers.Adam(
        learning_rate=tf.compat.v1.train.exponential_decay(
            learning_rate=learning_rate,
            global_step=tf.compat.v1.train.get_global_step(),
            decay_steps=10000,
            decay_rate=0.96)))
print("Training model...")
model.train(input_fn=lambda: input_fn(features=x_train, labels=y_train, training=True), steps=1000) # originally steps=1000 from template
print("Validate model...")
validation_results = model.evaluate(input_fn=lambda: input_fn(features=x_validation, labels=y_validation, training=False), steps=1)
print("Testing model...")
testing_results = model.evaluate(input_fn=lambda: input_fn(features=x_test, labels=y_test, training=False), steps=1)
end_timer = time.time()
print("Model training and testing took ", round(end_timer - timer_start, 1), " seconds.")
print("Validation results: ")
print(validation_results)
print("Testing results: ")
print(testing_results)
print("hidden_units=",hidden_units,"                                ")

Building model: Hidden units =  [37, 30, 19]
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\R-k-l\\AppData\\Local\\Temp\\tmpheb6cww1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000018ED65C9F98>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0,

In [8]:
validation_predictions = list(model.predict(input_fn=lambda: input_fn(features=x_validation, labels=y_validation, training=False), yield_single_examples=True))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\R-k-l\AppData\Local\Temp\tmpheb6cww1\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [9]:
validation_predictions

[{'logits': array([ 3.0641696, -0.8200993, -3.873588 ], dtype=float32),
  'probabilities': array([9.7892052e-01, 2.0129401e-02, 9.4998692e-04], dtype=float32),
  'class_ids': array([0], dtype=int64),
  'classes': array([b'0'], dtype=object),
  'all_class_ids': array([0, 1, 2]),
  'all_classes': array([b'0', b'1', b'2'], dtype=object)},
 {'logits': array([-5.6047826 , -0.38457188,  3.1437242 ], dtype=float32),
  'probabilities': array([1.5414867e-04, 2.8513363e-02, 9.7133249e-01], dtype=float32),
  'class_ids': array([2], dtype=int64),
  'classes': array([b'2'], dtype=object),
  'all_class_ids': array([0, 1, 2]),
  'all_classes': array([b'0', b'1', b'2'], dtype=object)},
 {'logits': array([ 7.78368 , -6.797496, -3.55606 ], dtype=float32),
  'probabilities': array([9.9998760e-01, 4.6501870e-07, 1.1890721e-05], dtype=float32),
  'class_ids': array([0], dtype=int64),
  'classes': array([b'0'], dtype=object),
  'all_class_ids': array([0, 1, 2]),
  'all_classes': array([b'0', b'1', b'2'], dt

In [10]:
validation_predictions_class_ids = [item['class_ids'][0] for item in validation_predictions]

In [11]:
print(validation_predictions_class_ids)

[0, 2, 0, 0, 2, 1, 0, 1, 2, 2, 2, 2, 2, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0]
